## Lending Club - Case Study

#### Business Understanding

   You work for a `consumer finance company` which specialises in lending various types of loans to urban customers. When the company receives a loan application, the company has to make a decision for loan approval based on the applicant’s profile. Two types of risks are associated with the bank’s decision:

`1. If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company`

`2. If the applicant is not likely to repay the loan, i.e. s/he is likely to default, then approving the loan may lead to a financial loss for the company`
 
The data given contains the information about past loan applicants and whether they `‘defaulted’` or not. The aim is to identify patterns which indicate if a person is likely to `default`, which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc.

In [ ]:
# Importing all libraries required.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# set it to convert scientific values
pd.options.display.float_format = '{:.2f}'.format 


In [ ]:
# Reading the provided .csv loan dataset file and converting the datatype into Object type, as Columns have mixed data types.
loan = pd.read_csv('../Lending Club- Case Study/loan.csv',dtype=object)


In [ ]:
# Rows and columns of the dataset
loan.shape

In [ ]:
loan.info() #Summary of the dataset

loan.dtypes #data-types of all the columns

#### Data Cleaning

In [ ]:
# checking whether there is any duplicate data in any column based on their ID
duplicate= loan[loan.duplicated('id')]
duplicate.count()

So, there is no duplicates in the entire Loan dataset.

In [ ]:
# Adding up the null values to check how many rows have missing values
print('Number of empty Rows:',   loan.isnull().all(axis=1).sum())

# Adding up the null values to check how many columns have missing values
print('Number of empty Columns:',    loan.isnull().all(axis=0).sum())

`Observation 1: There are no empty rows in the whole dataset.
 Observation 2: There are 54 columns have all missing values.`

In [ ]:
# to check Which columns have null values
loan.isnull().sum()

In [ ]:
# to Drop columns where all values are null (According to data,below columns have null values in sequence)
loan.drop(loan.iloc[:, 53:105], inplace = True, axis = 1) 

In [ ]:
loan

In [ ]:
pd.get_option("display.max_columns") #to know how much columns can be shown.

In [ ]:
pd.set_option("display.max_columns",None) #Now every column can be seen which is good for visualization

In [ ]:
loan.head()

In [ ]:
#Now Dropping additional Columns that we dont need anymore having null values
col_drop_1 = ["member_id","emp_title","url","desc","zip_code","mths_since_last_delinq","mths_since_last_record","next_pymnt_d","mths_since_last_major_derog","tax_liens","tot_hi_cred_lim","total_bal_ex_mort","total_bc_limit","total_il_high_credit_limit"]
loan.drop(labels = col_drop_1, axis =1, inplace=True)

In [ ]:
#checking the columns are now dropped or not which are having null values as mentioned above
loan.head()

In [ ]:
loan.shape   #so now after Data cleaning, we have 45 columns left 

In [ ]:
#To set the emp_length column to only number format for data analysis
loan.emp_length.fillna('0',inplace=True) #Setting all the null values of emp_length column to '0'

loan['emp_length']=loan.emp_length.str.extract('(\d+)')  #extracting all the Numbers from the emp_length column
loan.head()

In [ ]:
#Cleaning pub_rec_bankruptcies column for data analysis
print("Before data cleaning: ", loan.pub_rec_bankruptcies.isnull().sum())

loan.pub_rec_bankruptcies.fillna('Not Known',inplace=True) #Setting null values as 'Not Known'
print("After data cleaning: ",  loan.pub_rec_bankruptcies.isnull().sum())

In [ ]:
# To remove % symbol from 'int_rate' and 'revol_util' columns for future calculations ,using .strip()
loan['int_rate'] = loan['int_rate'].str.strip('%')

loan['revol_util'] = loan['revol_util'].str.strip('%')

In [ ]:
loan.head(2)

In [ ]:
# converting all the 'amount' columns into 'numeric data' to analyze any relation between them
col1 = ['loan_amnt','funded_amnt','int_rate','funded_amnt_inv','installment','annual_inc','dti','emp_length','total_pymnt'] #listing out all the columns having amounts, for the conversion
loan[col1] = loan[col1].apply(pd.to_numeric)

In [ ]:
# percentages of important variable loan status in total loans issued
(loan.loan_status.value_counts()*100)/len(loan)


`Observations` <br>
_1. 82.96 percent loans were fully paid._ <br>
_2. Close to 14% loans were charged off_

In [ ]:
#Percentages of the variable 'purpose of loan' as per total loans issued
(loan.purpose.value_counts()*100)/len(loan)

`Observations` <br>
_1. Most of the loans are taken for 'debt consolidation' i.e. 47% and 'Credit card bill payment' i.e. 13% ._ <br>
_2. 'Other' is also an important purpose of loan taken i.e. 10% ._

#### Deriving Columns for Data Analysis

In [ ]:
#Deriving new columns Month and Year, using existing columns such as issue date 'issue_d'

loan.issue_d = pd.to_datetime(loan.issue_d, format='%b-%y') #converting to Datetime format to derive new columns
loan['year']=loan['issue_d'].dt.year #new Year column has been added to loan dataset
loan['month']=loan['issue_d'].dt.month  #new Month column has been added to loan dataset

loan.head(2)

`Observations` <br>
_Two new columns Month and Year were derived from issue date column._


#### Univariate Analysis - 

In [ ]:
# Basic statistical details of loan_amnt column using describe method - Quantitative Variables
loan['loan_amnt'].describe()

In [ ]:
# Ignore warnings due to version problems
import warnings
warnings.filterwarnings('ignore')

#Plotting loan amount column using boxplot
sns.boxplot(loan.loan_amnt)

In [ ]:
# Basic statistical details of total_pymnt column using describe method - Quantitative Variables
loan['total_pymnt'].describe()

In [ ]:
#Plotting total payment column using boxplot
sns.boxplot(loan.total_pymnt)

In [ ]:
# Basic statistics details of annual_inc column - Quantitative Variables
print('Before the removal of Outliers :',  loan['annual_inc'].describe())

In [ ]:
# Removing the outliers using 0.99 quantile
q= loan["annual_inc"].quantile(0.99)   # For each of the dataframe Column we can use:

#Filter with
loan=loan[loan["annual_inc"]<q]

print('After the removal of Outliers :',  loan['annual_inc'].describe())

In [ ]:
#Plotting annual_inc column using boxplot
sns.boxplot(loan.annual_inc)

In [ ]:
# Basic statistics details of interest rate column - Quantitative Variables
loan['int_rate'].describe()

_As we can see, the average interest rate is **12%** ._ <br>
_After 75% quartile, interest rate has increased to **25%** from 15%_

In [ ]:
#Plotting the int_rate column
sns.boxplot(loan['int_rate'])
plt.title('Interest Rate Box Plot', fontsize = 15)
plt.xlabel('Interest Rate', fontsize = 12)
plt.show()

#### Correlation Matrix - Quantitative Variables :

_Bivariate analysis - Finding the Correlation matrix_

In [ ]:
# Plotting the Heatmap for the correlation matrix to see relation between different variables
loan_corr = loan.corr() #to find the relation between each column in the dataset

sns.set(font_scale=1.1) #changing the font size according to requirement

plt.figure(figsize=[17,17]) #how big you want the figure to be
sns.heatmap(loan_corr, cmap="Blues", fmt='.2%', annot=True) #plotting of the correlation matrix using seaborn heatmap
plt.show()

`Observation:` <br>
_1. Loan amount, investor amount, funding amount are strongly correlated_  <br>
_2. Annual income with DTI(Debt-to-income ratio) is correlated negatively(that is if annual_income is high, dti(Debt-to-income) ratio is low & vice versa)_  <br>
_3. Positive correlation between annual income and employment years, that means the income increases with work experience_

In [ ]:
# Dropping other columns which are not needed for further analysis:
col_drop_4 = ["application_type","policy_code","initial_list_status","installment","pymnt_plan"]
loan.drop(labels = col_drop_4, axis =1, inplace=True)
print(loan.shape)
loan.isnull().sum()

#### Derive Columns For Analysis

In [ ]:
#Deriving new columns using some existing column
# to categorise the loan_amount into ranges for bivariate analysis in future:
loan['loan_amnt_cat'] = pd.cut(loan['loan_amnt'], [0, 7000, 14000, 21000, 28000, 35000], labels=['0-7000', '7000-14000', '14000-21000', '21000-28000', '28000 +'])

# to categorise the annual_income into ranges for bivariate analysis in future:
loan['annual_inc_cat'] = pd.cut(loan['annual_inc'], [0, 20000, 40000, 60000, 80000,1000000], labels=['0-20000', '20000-40000', '40000-60000', '60000-80000', '80000 +'])

# to categorise the interest_rates into ranges for bivariate analysis in future:
loan['int_rate_cat'] = pd.cut(loan['int_rate'], [0, 10, 12.5, 16, 20], labels=['0-10', '10-13', '12.5-16', '16 +'])

# to categorise dti into ranges for bivariate analysis in future:
loan['dti_cat'] = pd.cut(loan['dti'], [0, 5, 10, 15, 20, 25], labels=['0-5', '05-10', '10-15', '15-20', '25+'])


In [ ]:
#checking the rows, columns and the data
print(loan.shape)
loan.head(1)

`Observation` _is we have created extra 5 columns after dropping many_

#### Univariate Analysis

In [ ]:
# checking the distribution of the three loan amount fields(loan_amnt, funded_amnt and funded_amnt_inv),
# using distribution plots - Quantitative Variables

plt.figure(figsize=(15,8),facecolor='c')
sns.set_style("whitegrid")

# subplot 1
plt.subplot(2, 3, 1)
ax = sns.distplot(loan['loan_amnt'],rug = True)
ax.set_title('Loan Amount - Distribution Plot',fontsize=14,color='w')
ax.set_xlabel('Loan Amount',fontsize=12,color='w')

# subplot 2
plt.subplot(2, 3, 2)
ax = sns.distplot(loan['funded_amnt'],rug = True)
ax.set_title('Funded Amount - Distribution Plot',fontsize=14,color='w')
ax.set_xlabel('Funded Amount',fontsize=12,color='w')

# subplot 3
plt.subplot(2, 3, 3)
ax = sns.distplot(loan['funded_amnt_inv'],rug = True)
ax.set_title('Funded Amount Inv. - Distribution Plot',fontsize=14,color='w')
ax.set_xlabel('Funded Amount Inv.',fontsize=12,color='w')
plt.show()

`Observation:` <br>
_1. Distribution of amounts for all three looks very much similar._<br>
_2. We will work with only loan amount column for rest of our analysis._

In [ ]:
# Univariate Analysis using 'Loan amount'  -  Quantitative Variables

plt.figure(figsize=(15,8),facecolor='c')
sns.set_style("whitegrid")

# subplot 1 - using dist. Plot
plt.subplot(2, 2, 1)
ax = sns.distplot(loan['loan_amnt'],rug = True)
ax.set_title('Loan Amount - Distribution Plot',fontsize=16,color='w')
ax.set_xlabel('Loan Amount',fontsize=14,color='w')

# subplot 2 - using Boxplot
plt.subplot(2, 2, 2)
ax = sns.boxplot(y=loan['loan_amnt'])
ax.set_title('Loan Amount - Box Plot',fontsize=16,color='w')
ax.set_ylabel('Loan Amount',fontsize=14,color='w')
plt.show()

`Observations :` <br>
_From both the plots above, we can know that most of the loan_amounts are in range of 5000 - 15000_

In [ ]:
# Univariate Analysis using 'Intrest Rate'  -  Quantitative Variables

plt.figure(figsize=(15,8),facecolor='c')
sns.set_style("whitegrid")

# subplot 1
plt.subplot(2, 2, 1)
ax = sns.distplot(loan['int_rate'],rug = True)
ax.set_title('Interest Rate - Distribution Plot',fontsize=16,color='w')
ax.set_xlabel('Interest Rate',fontsize=14,color='w')

# subplot 2
plt.subplot(2, 2, 2)
ax = sns.boxplot(y=loan['int_rate'])
ax.set_title('Interest Rate - Box Plot',fontsize=16,color='w')
ax.set_ylabel('Interest Rate',fontsize=14,color='w')
plt.show()

`Observations :` <br>
_From both the plots above, we can know that, most of the Interest Rates on loans are in range of 10% - 15%_

In [ ]:
# Univariate Analysis using 'Annual Income' column - Quantitative Variables

plt.figure(figsize=(15,8),facecolor='c')
sns.set_style("whitegrid")

# subplot 1
plt.subplot(2, 2, 1)
ax = sns.distplot(loan['annual_inc'],rug = True)
ax.set_title('Annual Income - Distribution Plot',fontsize=16,color='w')
ax.set_xlabel('Annual Income',fontsize=14,color='w')

# subplot 2
plt.subplot(2, 2, 2)
plt.title('Annual Income - Box Plot')
ax = sns.boxplot(y=loan['annual_inc'])
ax.set_title('Annual Income - Box Plot',fontsize=16,color='w')
ax.set_ylabel('Annual Income',fontsize=14,color='w')
plt.show()

`Observations :`<br>
_The above 2 plots show that, most of the borrower's Annual incomes are in the range of 40000 - 80000._

In [ ]:
# Univariate Analysis - Unordered Categorical Variables - Loan Status

plt.figure(figsize=(10,8),facecolor='c')
sns.set_style("whitegrid")

ax = sns.countplot(x="loan_status",data=loan,palette='mako')
ax.set_title('Loan Status',fontsize=14,color='w')
ax.set_xlabel('Loan Status',fontsize=14,color = 'w')
ax.set_ylabel('Loan Application Count',fontsize=14,color = 'w')

# To show count of values above bars
s=loan['loan_status'].value_counts()
for i, v in s.reset_index().iterrows():
    ax.text(i, v.loan_status + 0.3 , v.loan_status, color='k')

`Observations :` <br>
_The above plot shows that close to 14% loans were charged off out of total loan issued._

In [ ]:
# Univariate Analysis - Unordered Categorical Variables - Purpose Of Loan

plt.figure(figsize=(14,8),facecolor='c')
sns.set_style("whitegrid")

ax = sns.countplot(y="purpose",data=loan,hue='loan_status',palette='GnBu_d')
ax.set_title('Purpose Of Loan Plot',fontsize=14,color='w')
ax.set_ylabel('Purpose Of Loan',fontsize=14,color = 'w')
ax.set_xlabel('Loan Application Count',fontsize=14,color = 'w')      
plt.show()

`Observations :` <br>
_The above plot shows that, most of the loans were taken for the **purpose of debt consolidation & paying credit card bill**._ <br>
_Number of charged off count is also __high__ too for these loans._

In [ ]:
# Univariate Analysis - Unordered Categorical Variables - Home Ownership

plt.figure(figsize=(10,6),facecolor='c')

ax = sns.countplot(x="home_ownership",data=loan,hue='loan_status',palette='GnBu_d')
ax.legend(bbox_to_anchor=(1, 1)) #to put the legend on upper right loc
ax.set_title('Home Ownership Plot',fontsize=14,color='w')
ax.set_xlabel('Home Ownership',fontsize=14,color = 'w')
ax.set_ylabel('Loan Application Count',fontsize=14,color = 'w')      
plt.show()

`Observations :` <br>
_The above plot shows that, most of them living in __rented home__ or in __mortgaged home__._ <br>
_Applicant numbers are high from these 2 categories so charged-off is high too._

In [ ]:
# Derived Column - Ordered Categorical Variables

# Checking the number of loans which were approved in every month/year

plt.figure(figsize=(14,6),facecolor='c')
loan.groupby(['year','month']).id.count().plot(kind='bar')  # We will use derived column year to check pattern of loan issuing over the years.

plt.title("No of loan applications issued over the years:",fontsize=14,color='w')
plt.ylabel('Loan applications count',fontsize=14,color='w')
plt.xlabel('Year,Month',fontsize=14,color = 'w')      

plt.show()

`Observations: ` <br>
_From the above Plot, count of loan application is __increasing__ every passing year._ <br>
_So increase in number of loan applications are adding more to number of charged off applications. And number of loans issued in 2008( May-October) was dropped due to issues like recesion perhaps._

In [ ]:
# Univariate Analysis - Ordered Categorical Variables- Loan Paying Term

plt.figure(figsize=(10,6),facecolor='c')

ax = sns.countplot(x="term",data=loan,hue='loan_status',palette='GnBu_d')
ax.set_title('Loan Paying Term',fontsize=14,color='w')
ax.set_xlabel('Loan Repayment Term',fontsize=14,color = 'w')
ax.set_ylabel('Loan Application Count',fontsize=14,color = 'w')      
ax.legend(bbox_to_anchor=(1, 1))
plt.show()

`Observations :` <br>
_The above plot shows that, applicants who took loan to repay in __60 months__ had more % of number of applicants getting 
 __charged off__ as compared to applicants who had taken loan for __36 months__._
 
 #### Bivariate Analysis : Part 1

`In this analysis, we will analyze the loan status against some important columns which might have played important role 
in 'charged off' of loans. Lets try to find proportion of charged offs in some categories.` <br>

_To calculate the proportion of charged off loans, will follow below steps:_ <br>

_1. Group loans by different variables and loan status, get the count,use .unstack() to return a DataFrame._<br>
_2. Since we’re going to use this for further analysis, it will be helpful to use .reset_index() to clean up the index._<br>
_3. Assign the new DataFrame to a variable._<br>
_4. Create a simple derived column that sums charged off,current and fully paid loans for each category._<br>
_5. Divide the number of charged off loans by the total number of loans to get the proportion of charged off loans._<br>
_6. Store this as another derived column._<br>
_7. Finally, output the whole table, sorted by proportion of charged off in each category issued loans._<br>

In [ ]:
# Bivariate Analysis - annual income against Chargedoff_Proportion -

inc_range_vs_loan = loan.groupby(['annual_inc_cat', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
inc_range_vs_loan['Total'] = inc_range_vs_loan['Charged Off'] + inc_range_vs_loan['Current'] + inc_range_vs_loan['Fully Paid'] 
inc_range_vs_loan['Chargedoff_Proportion'] = inc_range_vs_loan['Charged Off'] / inc_range_vs_loan['Total']
inc_range_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

`Observations:` <br>
_1. Income range of __80000 above__ has __less chances__ of loans going charged off._<br>
_2. Income range of __0-20000__ has __high chances__ of loans going charged off._<br>
_3. Notice that with __increase__ in annual income, charged off proportion get __decreased__._

In [ ]:
# To draw bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(12, 8),facecolor='c')

ax1.set_title('Annual Income vs Chargedoff Proportion',fontsize=15,color = 'w')
ax1=sns.barplot(x='annual_inc_cat', y='Chargedoff_Proportion', data=inc_range_vs_loan)
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'w')
ax1.set_xlabel('Annual Income',fontsize=14,color='w')
plt.show()

`Observations:` _(same as above)_<br>
_1. Income range of __80000 above__ has __less chances__ of loans going charged off._<br>
_2. Income range of __0-20000__ has __high chances__ of loans going charged off._<br>
_3. With __increase__ in annual income, charged off proportion get __decreased__._

In [ ]:
# Bivariate Analysis - purpose of Loan against Chargedoff_Proportion:

purpose_vs_loan = loan.groupby(['purpose', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
purpose_vs_loan['Total'] = purpose_vs_loan['Charged Off'] + purpose_vs_loan['Current'] + purpose_vs_loan['Fully Paid'] 
purpose_vs_loan['Chargedoff_Proportion'] = purpose_vs_loan['Charged Off'] / purpose_vs_loan['Total']
purpose_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

`Observations :` <br>
_* **small Business** applicants have **high chances** of getting charged off._<br>
_* __renewable_energy__ where charged off proportion is __better__ as compare to other categories._

In [ ]:
# Now, drawing bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(14, 8),facecolor='c')

ax1.set_title('Purpose of Loans vs Chargedoff Proportion',fontsize=15,color = 'w')
ax1=sns.barplot(y='purpose', x='Chargedoff_Proportion', data=purpose_vs_loan)
ax1.set_ylabel('Purpose of Loans',fontsize=14,color='w')
ax1.set_xlabel('Chargedoff Proportion',fontsize=14,color = 'w')
plt.show()

`Observations :` <br>
_* **small Business** applicants have **high chances** of getting charged off._<br>
_* __renewable_energy__ where charged off proportion is __better__ as compare to other categories._

In [ ]:
# Bivariate Analysis -  grade against Chargedoff_Proportion

grade_vs_loan = loan.groupby(['grade', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
grade_vs_loan['Total'] = grade_vs_loan['Charged Off'] + grade_vs_loan['Current'] + grade_vs_loan['Fully Paid'] 
grade_vs_loan['Chargedoff_Proportion'] = grade_vs_loan['Charged Off'] / grade_vs_loan['Total']
grade_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

`Observations :` <br>
_* Grade "A" has very less chances of charged off._<br>
_* Grade "F" and "G" have very high chances of charged off._<br>
_* Chances of charged off is increasing with grade moving from "A" towards "G"_

In [ ]:
# Now, drawing bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(14, 8),facecolor='c')
ax1.set_title('Grades vs Chargedoff Proportion',fontsize=15,color='w')
ax1=sns.barplot(x='grade', y='Chargedoff_Proportion', data=grade_vs_loan)
ax1.set_xlabel('Grades',fontsize=14,color='w')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color ='w')
plt.show()

# Observations are same as mentioned above.

In [ ]:
# Checking the data about Grades - 
# Bivariate Analysis - 'sub_grade' against 'chargedoff_proportion'

grade_vs_loan = loan.groupby(['sub_grade', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
grade_vs_loan['Total'] = grade_vs_loan['Charged Off'] + grade_vs_loan['Current'] + grade_vs_loan['Fully Paid'] 
grade_vs_loan['Chargedoff_Proportion'] = grade_vs_loan['Charged Off'] / grade_vs_loan['Total']
grade_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

`Observations :` <br>
_* sub grade of __"A"__ has very __less chances__ of charged off._<br>
_* sub grade of **"F"** and **"G"** have very **high chances** of charged off._<br>
_* proportion of charged off is __increasing__ with sub grades moving from sub_grade "A" to "G"._<br>

In [ ]:
# Now, drawing bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(14, 8),facecolor='c')
ax1.set_title('Sub Grades vs Chargedoff Proportion',fontsize=15,color='w')
ax1=sns.barplot(x='sub_grade', y='Chargedoff_Proportion', data=grade_vs_loan)
ax1.set_xlabel('Sub Grades',fontsize=10,color='w')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'w')
plt.show()

# Observations are same as mentioned above.

In [ ]:
# Bivariate Analysis - interest rate against Chargedoff_Proportion

interest_vs_loan = loan.groupby(['int_rate_cat', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
interest_vs_loan['Total'] = interest_vs_loan['Charged Off'] + interest_vs_loan['Current'] + interest_vs_loan['Fully Paid'] 
interest_vs_loan['Chargedoff_Proportion'] = interest_vs_loan['Charged Off'] / interest_vs_loan['Total']
interest_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

In [ ]:
# Now, drawing bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(12, 8),facecolor='c')
ax1.set_title('Interest Rate vs Chargedoff Proportion',fontsize=15,color='w')
ax1=sns.barplot(x='int_rate_cat', y='Chargedoff_Proportion', data=interest_vs_loan)
ax1.set_xlabel('Interest Rate',fontsize=14,color='w')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'w')
plt.show()

`Observations :` <br>
_* interest rate less than 10% has very less chances of charged off. Intrest rates are starting from minimum 5 %._<br>
_* interest rate more than 16% has good chances of charged off as compared to other category interest rates._<br>
_* Charged off proportion is increasing with higher interest rates._<br>

In [ ]:
# Bivariate Analysis - employment length against Chargedoff_Proportion

emplen_vs_loan = loan.groupby(['emp_length', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
emplen_vs_loan['Total'] = emplen_vs_loan['Charged Off'] + emplen_vs_loan['Current'] + emplen_vs_loan['Fully Paid'] 
emplen_vs_loan['Chargedoff_Proportion'] = emplen_vs_loan['Charged Off'] / emplen_vs_loan['Total']
emplen_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

In [ ]:
# Now, drawing bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(12, 8),facecolor='c')
ax1.set_title('Employment Length vs Chargedoff Proportion',fontsize=15,color='w')
ax1=sns.barplot(x='emp_length', y='Chargedoff_Proportion', data=emplen_vs_loan)
ax1.set_xlabel('Employment Length',fontsize=14,color='w')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'w')
plt.show()

`Observations :` <br>
_* Those who are not working or have less than 1 year of work experience have high chances of getting charged off._ <br>
_* It makes sense as with less or no experience they don't have source of income to repay loan._ <br>
_* Rest of the applicants have more or less same chances of getting charged off._ <br>

In [ ]:
# Bivariate Analysis - address state against Chargedoff_Proportion

state_vs_loan = loan.groupby(['addr_state', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
state_vs_loan['Total'] = state_vs_loan['Charged Off'] + state_vs_loan['Current'] + state_vs_loan['Fully Paid'] 
state_vs_loan['Chargedoff_Proportion'] = state_vs_loan['Charged Off'] / state_vs_loan['Total']
state_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)


In [ ]:
# Now, drawing bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(20, 8),facecolor='c')
ax1.set_title('Address State vs Chargedoff Proportion',fontsize=15,color='w')
ax1=sns.barplot(x='addr_state', y='Chargedoff_Proportion', data=state_vs_loan)
ax1.set_xlabel('Address State',fontsize=14,color='w')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'w')
plt.show()

`Observations :`<br>
_* state **'NE'** has very high chances of charged off but number of applications are too low to make any decisions._<br>
_* **NV,AK and FL states** shows good number of charged offs in good number of applications._<br>

In [ ]:
# Bivariate Analysis - verification_status against Chargedoff_Proportion

verfy_vs_loan = loan.groupby(['verification_status', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
verfy_vs_loan['Total'] = verfy_vs_loan['Charged Off'] + verfy_vs_loan['Current'] + verfy_vs_loan['Fully Paid'] 
verfy_vs_loan['Chargedoff_Proportion'] = verfy_vs_loan['Charged Off'] / verfy_vs_loan['Total']
verfy_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)

In [ ]:
# Now, drawing bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(12, 8),facecolor='c')
ax1.set_title('Verification Status vs Chargedoff Proportion',fontsize=15,color='w')
ax1=sns.barplot(x='verification_status', y='Chargedoff_Proportion', data=verfy_vs_loan)
ax1.set_xlabel('Verification Status',fontsize=14,color='w')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'w')
plt.show()

`Observations : ` <br>
_* There is not much difference in charged off proportion._ <br>
_* This variable doesn't provide any insights for charged off._ <br>

In [ ]:
# Bivariate Analysis - Derogatory Public Record against Chargedoff_Proportion

bankruptcies_vs_loan = loan.groupby(['pub_rec', 'loan_status']).loan_status.count().unstack().fillna(0).reset_index()
bankruptcies_vs_loan['Total']=bankruptcies_vs_loan['Charged Off']+bankruptcies_vs_loan['Current']+bankruptcies_vs_loan['Fully Paid'] 
bankruptcies_vs_loan['Chargedoff_Proportion'] = bankruptcies_vs_loan['Charged Off'] / bankruptcies_vs_loan['Total']
bankruptcies_vs_loan.sort_values('Chargedoff_Proportion', ascending=False)


In [ ]:
# Now, drawing bar plots on the data calculated above, to visualize the pattern and to understand the data better:

fig, ax1 = plt.subplots(figsize=(12, 7),facecolor='c')
ax1.set_title('Derogatory Public Record vs Chargedoff Proportion',fontsize=15,color='w')
ax1=sns.barplot(x='pub_rec', y='Chargedoff_Proportion', data=bankruptcies_vs_loan)
ax1.set_xlabel('Derogatory Public Record',fontsize=14,color='w')
ax1.set_ylabel('Chargedoff Proportion',fontsize=14,color = 'w')
plt.show()

`Observations:`<br>
_* A derogatory item is an entry that may be considered negative by lenders because it indicates risk and hurts_ <br>
_your ability to qualify for credit or other services. Public records and collections are derogatory items_ <br>
_because they reflect financial obligations that were not paid as agreed._ <br>
_* Those who already have pub_rec value 1 or 2 have charged off chances higher than who have no Derogatory Public Record._ <br>
_* pub_rec count 3-4 has less numbers so cannot reach on any conclusions._


#### Bivariate Analysis - Part 2 - 

In [ ]:
# Bivariate Analysis - Puprose of loan vs Loan amount - Box Plot

plt.figure(figsize=(12,8),facecolor='c')
ax = sns.boxplot(y='purpose', x='loan_amnt', data =loan,palette='rainbow')
ax.set_title('Purpose of Loan vs Loan Amount',fontsize=15,color='w')
ax.set_ylabel('Purpose of Loan',fontsize=14,color = 'w')
ax.set_xlabel('Loan Amount',fontsize=14,color = 'w')
plt.show()


`Observations:`<br>
_* Median,95th percentile,75th percentile of loan amount is highest for loan taken for small business purpose among all purposes._<br>
_* Debt consolidation is second and Credit card comes 3rd._

In [ ]:
# Bivariate Analysis - Puprose of loan vs Loan amount -  using Violin Plot
# Violin plots show distribution of quantitative data across several levels of one (or more) categorical variables such that those distributions can be compared. 

plt.figure(figsize=(14,8),facecolor='c')
ax=sns.violinplot(y='purpose', x='loan_amnt', data =loan)
ax.set_title('Purpose of Loan vs Loan Amount',fontsize=15,color='w')
ax.set_ylabel('Purpose of Loan',fontsize=14,color = 'w')
ax.set_xlabel('Loan Amount',fontsize=14,color = 'w')
plt.show()

`Observations : `<br>
_* Loans taken for small business purpose, Debt consolidation and Credit card are somewhat evenly distributed as compared to<br>
 loan taken for other purposes._

In [ ]:
# Now, Bivariate Analysis - Term of loan vs Interest Rate - Box Plot

plt.figure(figsize=(10,6),facecolor='c')
ax = sns.boxplot(y='int_rate', x='term', data =loan,palette='rainbow')
ax.set_title('Term of loan vs Interest Rate',fontsize=15,color='w')
ax.set_ylabel('Interest Rate',fontsize=14,color = 'w')
ax.set_xlabel('Term of loan',fontsize=14,color = 'w')
plt.show()

`Observations : `<br>
_* It is clear that the average interest rate is higher for the __60 months__ term of loan._ <br>
_* Most loans issued for longer term had higher interest rates during repayment._

In [ ]:
# Bivariate Analysis - Purpose vs Interest Rate - Box Plot

plt.figure(figsize=(14,8),facecolor='c')
ax = sns.boxplot(x='int_rate', y='purpose', data =loan,palette='rainbow')
ax.set_title('Purpose vs Interest Rate',fontsize=15,color='w')
ax.set_xlabel('Interest Rate',fontsize=14,color = 'w')
ax.set_ylabel('Purpose of Loan',fontsize=14,color = 'w')
plt.show()

`Observations : `<br>
_* It is clear that avearge intrest rate is **highest** for **small business purpose**._ <br>
_* Loans taken for small business purposes had to repay the loan with more intrest rate as compared to other._ <br>
_* Debt consolidation is **2nd** where borrowers had to pay more interest rate._

In [ ]:
# Bivariate Analysis - Grade vs Interest Rate - Box Plot

plt.figure(figsize=(14,8),facecolor='c')
ax = sns.boxplot(y='int_rate', x='grade', data =loan,palette='rainbow',order = 'ABCDEFG')
ax.set_title('Grade vs Interest Rate',fontsize=15,color='w')
ax.set_ylabel('Interest Rate',fontsize=14,color = 'w')
ax.set_xlabel('Grade',fontsize=14,color = 'w')
plt.show()

`Observations : ` <br>
_* A-grade is a top letter grade for a lender to assign to a borrower._<br>
_* The higher the borrower's credit grade,the lower the interest rate offered to that borrower on a loan._<br>
_* It is clear that intrest rate is increasing with grades moving from A to F._

In [ ]:
# Now, Bivariate Analysis - Loan Amount vs Interest Rate - Box Plot

plt.figure(figsize=(14,8),facecolor='c')
ax = sns.boxplot(y='int_rate', x='loan_amnt_cat', data =loan,palette='rainbow')
ax.set_title('Loan amount vs Interest Rate',fontsize=15,color='w')
ax.set_ylabel('Interest Rate',fontsize=14,color = 'w')
ax.set_xlabel('Loan Amount',fontsize=14,color = 'w')
plt.show()

`Observations : `<br>
_* It is clear that intrest rate is increasing with loan amount increase._<br>
_* probably when loan amount is more it is taken for longer loan term, we saw earlier that longer the loan term more the interest rate._

In [ ]:
# Bivariate Analysis - DTI vs Interest Rate - Box Plot

plt.figure(figsize=(14,8),facecolor='c')
ax = sns.boxplot(y='int_rate', x='dti_cat', data =loan,palette='rainbow')
ax.set_title('Debt to Income Ratio v/s Interest Rate',fontsize=15,color='w')
ax.set_ylabel('Interest Rate',fontsize=14,color = 'w')
ax.set_xlabel('Debt to Income Ratio',fontsize=14,color = 'w')
plt.show()

`Observations : ` <br>
_* If your DTI is low enough you may get a lower interest rate._ <br>
_* Plot shows no significant variation but there is __slight increase__ in __interest rate__ with __increase in DTI__._

In [ ]:
# Bivariate Analysis - Recoveries vs Annual income - Box Plot

loan['recoveries']=pd.to_numeric(loan['recoveries'],errors='coerce')
loan_rec=loan[loan['recoveries'] > 0.00]

# calculate recovery percentage 
loan_rec['recovery_prcnt']=((loan_rec['recoveries']/loan_rec['loan_amnt']) * 100) 

# plot
plt.figure(figsize=(14,9),facecolor='c')
ax=sns.boxplot(x='recovery_prcnt',y='annual_inc_cat',data=loan_rec,palette='rainbow')
ax.set_title('Percentage of Loan Amount Recovered v/s Annual Income',fontsize=15,color = 'w')
ax.set_ylabel('Annual income',fontsize=14,color = 'w')
ax.set_xlabel('Percentage of loan amount recovered post Charged off',fontsize=14,color = 'w')
plt.show()

`Observations : ` <br>
_* Higher percentage of __loan amount__ is recovered when __annual income is high__._ <br>
_* Plot shows __no significant variation__ but there is __slight increase in recovery percentage__ with increase in __annual income__._

In [ ]:
# Bivariate analysis - Bar Plot to see annual income variation across grade for every loan status:

plt.figure(figsize=(10,6), dpi=80, facecolor='c', edgecolor='k')
ax=sns.barplot(x='grade', y='annual_inc', hue="loan_status", data=loan, estimator=np.mean,order='ABCDEFG')
ax.set_title('Grade vs Annual income',fontsize=15,color='w')
ax.set_ylabel('Annual income',fontsize=14,color = 'w')
ax.set_xlabel('Grades',fontsize=14,color = 'w')
plt.show()

`Observations : ` <br>
_* From this we can conclude that the ones getting **'charged off'** have lower annual incomes than the ones 
who'paid fully' <br>for each and every grade (i.e. at same interest range)_

In [ ]:
# Bivariate Analysis - Year vs Interest Rate - Box Plot

plt.figure(figsize=(14,8),facecolor='c')
ax = sns.boxplot(y='int_rate', x='year', data =loan,palette='rainbow')
ax.set_title('Year vs Interest Rate',fontsize=15,color='w')
ax.set_ylabel('Interest Rate',fontsize=14,color = 'w')
ax.set_xlabel('Year',fontsize=14,color = 'w')
plt.show()

`Observations:` <br>
_* Plot shows __interest rate__ is increasing slowly with increase in __year__._

In [96]:
loan.shape

(39319, 46)